In [2]:
# !pip install transformers

     ---------------------------------------- 4.2/4.2 MB 22.2 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 69.6 MB/s eta 0:00:00
     ---------------------------------------- 86.2/86.2 kB 5.1 MB/s eta 0:00:00


In [5]:
# !pip install fastai

     -------------------------------------- 197.9/197.9 kB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 60.6/60.6 kB 3.2 MB/s eta 0:00:00
     --------------------------------------- 11.7/11.7 MB 65.5 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 63.9 MB/s eta 0:00:00
     ---------------------------------------- 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ------------------------------------- 448.4/448.4 kB 29.2 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 62.6 MB/s eta 0:00:00
     ---------------------------------------- 112.4/112.4 kB ? eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 69.9 MB/s eta 0:00:00
     ---------------------------------------- 181.6/181.6 kB ? eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.0
    Uninstalling pydantic-1.9.0:
      Successfully uninstalled pydantic-1.9.0


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings


In [7]:
from typing import Optional
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from fastai.text.all import *
import fastai
import re

# KOGPT 구현

In [9]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")
text = """ 삼성전자 주가 알려줘 """
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True
                        )
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


삼성전자 주가 알려줘 줬다는 의혹을 받고 있다.
검찰은 이 부회장이 박 전 대통령과 독대하면서 삼성이 최씨 측에 건넨 돈의 사용처를 캐물었다.
이날 오전 9시30분께 서울중앙지검에 출석한 최씨는 "최씨가 (박근혜) 대통령과의 단독면담을 요청했다"고 진술했다.
그는 또 "이번 사안은 대통령의 국정농단 사건"이라며 관련 혐의를 부인했다.</d> 지난해 12월 31일 오후 2시, 경기도 성남시의 한 아파트에서 불이 나 주민 3명이 숨지고 1명이 다쳤다.
불은 집 안에 있던 김모(53·여), 정모씨(50)를 비롯해


In [ ]:
with open('tale.txt') as f:
    lines = f.read()
lines=" ".join(lines.split())

#model input output tokenizer
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

#split data
train=lines[:int(len(lines)*0.9)]
test=lines[int(len(lines)*0.9):]
splits = [[0],[1]]

#init dataloader
tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
batch,seq_len = 8,256
dls = tls.dataloaders(bs=batch, seq_len=seq_len)

In [ ]:
#gpt2 ouput is tuple, we need just one val
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]
      
      
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()
lr=learn.lr_find()
print(lr)
learn.fine_tune(6)

In [ ]:
#gpt2 ouput is tuple, we need just one val
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]
      
      
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()
lr=learn.lr_find()
print(lr)
learn.fine_tune(6)

# Streamlit 구현

In [1]:
# !pip install streamlit-chat

     ---------------------------------------- 1.2/1.2 MB 12.3 MB/s eta 0:00:00
     ---------------------------------------- 9.1/9.1 MB 64.6 MB/s eta 0:00:00
     ---------------------------------------- 111.5/111.5 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 17.9/17.9 MB 65.5 MB/s eta 0:00:00
     ------------------------------------- 232.0/232.0 kB 13.9 MB/s eta 0:00:00
     ---------------------------------------- 164.8/164.8 kB ? eta 0:00:00
     ------------------------------------- 812.8/812.8 kB 25.1 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 67.8 MB/s eta 0:00:00
     ------------------------------------- 181.2/181.2 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 63.1/63.1 kB ? eta 0:00:00
     -----------

In [2]:
import streamlit as st
from streamlit_chat import message

ImportError: cannot import name 'Literal' from 'typing' (C:\Users\bitcamp\Anaconda3\envs\nlp\lib\typing.py)

In [3]:
message("My message") 
message("Hello bot!", is_user=True)

2022-06-16 16:44:00.550 
  command:

    streamlit run C:\Users\bitcamp\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [4]:
import streamlit as st
from streamlit_chat import message
import requests

st.set_page_config(
    page_title="Streamlit Chat - Demo",
    page_icon=":robot:"
)

API_URL = "https://api-inference.huggingface.co/models/facebook/blenderbot-400M-distill"
headers = {"Authorization": st.secrets['api_key']}

st.header("Streamlit Chat - Demo")
st.markdown("[Github](https://github.com/ai-yash/st-chat)")

if 'generated' not in st.session_state:
    st.session_state['generated'] = []

if 'past' not in st.session_state:
    st.session_state['past'] = []

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

def get_text():
    input_text = st.text_input("You: ","Hello, how are you?", key="input")
    return input_text 


user_input = get_text()

if user_input:
    output = query({
        "inputs": {
            "past_user_inputs": st.session_state.past,
            "generated_responses": st.session_state.generated,
            "text": user_input,
        },"parameters": {"repetition_penalty": 1.33},
    })

    st.session_state.past.append(user_input)
    st.session_state.generated.append(output["generated_text"])

if st.session_state['generated']:

    for i in range(len(st.session_state['generated'])-1, -1, -1):
        message(st.session_state["generated"][i], key=str(i))
        message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Final_Project\\.streamlit\\secrets.toml'

In [5]:
from chatterbot import chatbot
from chatterbot.trainers import ListTrainer
 
#creating a new chatbot
chatbot = Chatbot('Edureka')
trainer = ListTrainer(chatbot)
trainer.train([ 'hi, can I help you find a course', 'sure I'd love to find you a course', 'your course have been selected'])
 
#getting a response from the chatbot
response = chatbot.get_response("I want a course")
print(response)


SyntaxError: invalid syntax (Temp/ipykernel_3348/2085097616.py, line 7)

In [4]:
# !pip install Literal

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\bitcamp\Anaconda3\envs\nlp\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\bitcamp\\AppData\\Local\\Temp\\pip-install-nwmw2pvj\\literal_92226daa06c046899fa23ed04ffce1df\\setup.py'"'"'; __file__='"'"'C:\\Users\\bitcamp\\AppData\\Local\\Temp\\pip-install-nwmw2pvj\\literal_92226daa06c046899fa23ed04ffce1df\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\bitcamp\AppData\Local\Temp\pip-pip-egg-info-dmin81fy'
         cwd: C:\Users\bitcamp\AppData\Local\Temp\pip-install-nwmw2pvj\literal_92226daa06c046899fa23ed04ffce1df\
    Complete output (1 lines):
    error in literal setup command: use_2to3 is invalid.
    ------------------

In [5]:
# !pip install streamlit

In [6]:
# !pip install streamlit-chat

In [7]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

import streamlit as st
from streamlit_chat import message as st_message

ImportError: cannot import name 'Literal' from 'typing' (C:\Users\bitcamp\Anaconda3\envs\nlp\lib\typing.py)